In [8]:
from spark_session import get_spark_session
from scripts.pyspark_window import do_exercice
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

In [9]:
# Create Spark session
spark_session = get_spark_session("Test Jupyter") 

In [10]:
df = spark_session.read_dataframe("Test/data.csv", header=True, inferSchema=True)

# Apply exercice
df = do_exercice(df)

# Display result
df.show()

+----+-------+------------+--------------------+-------------------+-------------------+
|year|quarter|revenue_rate|         rate_change|     cumulative_avg|         moving_avg|
+----+-------+------------+--------------------+-------------------+-------------------+
|2023|     Q1|        0.15|                NULL|               0.15|               0.15|
|2023|     Q2|        0.17|0.020000000000000018|               0.16|               0.16|
|2023|     Q3|        0.19| 0.01999999999999999|               0.17|               0.18|
|2023|     Q4|        0.22|                0.03|             0.1825|0.20500000000000002|
|2024|     Q1|        0.23|                NULL|               0.23|               0.23|
|2024|     Q2|        0.21|-0.02000000000000...|               0.22|               0.22|
|2024|     Q3|        0.24|                0.03|0.22666666666666666|0.22499999999999998|
+----+-------+------------+--------------------+-------------------+-------------------+



In [11]:
spark_session.stop()